In [1]:
from datetime import datetime
import dask.array as da
import os
import time
import h5py
import numpy as np
import xarray as xr
import downscale as down
import matplotlib.pyplot as plt

In [2]:
# down.matplotlib_update_settings()

In [3]:
def extract_sample_data(inputfile, outputfile,
                        clat=34.875, clon=-98.125, npix=3, dx=0.25):
    '''------------------------------------------------------------------------
    Extract a given bounding box centered in (clat, clon) with size npix
    grid cells from an input hdf file
    and save it as nc dataset for further analysis
    ------------------------------------------------------------------------'''
    # f = h5py.File(os.path.join(cdat.tmpa_dir, 'data_tmpa_3h.hdf5'), "r")
    f = h5py.File(inputfile, "r")
    # print(list(f.keys()))
    # print(list(f.attrs()))
    tmpalat = f['lat'][:]
    tmpalon = f['lon'][:]
    dates_int = f['dates'][:]
    hours_int = f['hours'][:]
    dset = f['prcp']
    print('dataset shape = {}'.format(dset.shape)) # too large to fit in memory!
    x = da.from_array(dset, chunks=(6, 6, 300))
    # UTC time
    dates = [datetime.strptime(str(integd)+str(inthour), '%Y%m%d%H')
                    for integd, inthour in zip(dates_int, hours_int)]
    # create xarray
    xrs0 = xr.DataArray(x,  coords={'lon':tmpalon, 'lat':tmpalat, 'time':dates},
                                                    dims=('lon', 'lat', 'time'))
    # set negative values to NaN (missing values)
    xrs = xrs0.where(xrs0 >= -0.001)
    # now extract a bounding box of interest:
    # clat = 34.875
    # clon = -98.125
    # dx = 0.25
    # npix = 3
    buffer = 0.50*npix*dx
    eps = 1e-4 # to make sure to include boundaires -> add an eps buffer
    # eps = 0 # to make sure to include boundaires -> add an eps buffer
    solat = clat - buffer + eps
    nolat = clat + buffer + eps
    ealon = clon + buffer + eps
    welon = clon - buffer + eps
    bcond = np.logical_and(
                np.logical_and( xrs.lat > solat, xrs.lat < nolat),
                np.logical_and( xrs.lon > welon, xrs.lon < ealon))
    # XARRAY WITH VALUES LOADED IN MEMORY
    box_3h = xrs.where(bcond, drop = True).load()
    print(box_3h.shape)
    box_3h.to_netcdf(outputfile)

In [4]:
extract_box = True # do it only once
# inputfile = os.path.join('..', cdat.tmpa_dir, 'data_tmpa_3h.hdf5')
tmpa_dir = os.path.join('..', 'data', 'tmpa_conus_data')
inputfile = os.path.join(tmpa_dir, 'data_tmpa_3h.hdf5')
outputdir = os.path.join('..', 'data', 'sample_data')
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
# outputfile = os.path.join('..', 'sample_data','okla.nc')
outputfile = os.path.join(outputdir,'okla.nc')
clat = 34.875
clon = -98.125
dx = 0.25
npix = 4
if extract_box:
    extract_sample_data(inputfile, outputfile,
                        clat=clat, clon=clon, npix=npix,dx=dx)

dataset shape = (280, 112, 2472)
(4, 4, 2472)


In [5]:
# box_3h = xr.open_dataset(outputfile)
box_3h = xr.open_dataarray(outputfile)
Tr = 100

res = {}  # initialize dictionary for storing results

In [6]:
box_3h

<xarray.DataArray 'array-bc01ef4f0d88ed05056fed05542b0c92' (lon: 4, lat: 4,
                                                            time: 2472)> Size: 158kB
[39552 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 16B -98.38 -98.12 -97.88 -97.62
  * lat      (lat) float32 16B 34.62 34.88 35.12 35.38
  * time     (time) datetime64[ns] 20kB 2007-12-27T03:00:00 ... 2018-10-31T21...

In [7]:
# downscaling:
print('Example: running the downscaling function')
init_time = time.time()
downres = down.downscale(box_3h, Tr, thresh=1.0, L0=0.0001, toll=0.005,
                        acf='mar', save_yearly=True,
                        maxmiss=36, clat=clat, clon=clon,
                        opt_method='genetic', plot=True)
end_time = time.time()
exec_time = end_time - init_time
print('Downscaling, execution time was = {} minutes'.format(exec_time/60))

Example: running the downscaling function
downscale warning: at least one box size has even length
Downscaling Intermittency


/home/arturo/anaconda3/envs/AXE/lib/python3.12/site-packages/xarray/core/groupby.py:668: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index_grouper = pd.Grouper(
/home/arturo/anaconda3/envs/AXE/lib/python3.12/site-packages/xarray/core/groupby.py:668: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index_grouper = pd.Grouper(
/home/arturo/anaconda3/envs/AXE/lib/python3.12/site-packages/xarray/core/groupby.py:668: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index_grouper = pd.Grouper(
/home/arturo/anaconda3/envs/AXE/lib/python3.12/site-packages/xarray/core/groupby.py:668: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index_grouper = pd.Grouper(
/home/arturo/anaconda3/envs/AXE/lib/python3.12/site-packages/xarray/core/groupby.py:668: FutureWarning: 'H' is deprecated and will b

Downscaling the correlation
Computing the correlation
differential_evolution step 1: f(x)= 0.006911770229444236
differential_evolution step 2: f(x)= 0.006911770229444236
differential_evolution step 3: f(x)= 0.006911770229444236
Polishing solution with 'L-BFGS-B'
integration domain is zero
Downscaling pdf - global Weibull parameters
[64.]
[13.1583309]
[0.85658344]
Downscaling pdf - yearly Weibull parameters
[]
[]
[]


/home/arturo/anaconda3/envs/AXE/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/arturo/anaconda3/envs/AXE/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


UnboundLocalError: cannot access local variable 'fval' where it is not associated with a value